<a href="https://colab.research.google.com/github/BhojRajBist/BhojRajBist/blob/main/2_netCDF_TIFF_Plot_Animate_COG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NCMRWF to TIFF to COG (Manual Download)

This code:
1. Convert the NetCDF to TIFF, choose best 121 hrs files, plot and animate
2. Convert TIFF to COG

### USER INPUT HERE

In [2]:
import os

root_dir = "20240621"

# Create the directory if it does not exist
if not os.path.exists(root_dir):
    os.makedirs(root_dir)
    print(f"Directory {root_dir} created.")
else:
    print(f"Directory {root_dir} already exists.")


Directory 20240621 already exists.


In [3]:
#Mention Root Directory
root_dir = "20240621"

In [4]:
# Define the latitude and longitude bounds for Nepal (For specific region change here)
# NCUMGLB12.5 only available for lat [26, 31] and lon [79, 89]

lat_bounds = [26, 31]
lon_bounds = [79, 89]

# Set the interval time in milliseconds
interval_time = 1000  # 1 second

### DON'T CHANGE BELOW THIS LINE

just run each cell below for the objective defined by the comment

#### STEP- 0: Initialization (Libraries, pacakages and inputs)

In [5]:
# Install necessary libraries
!pip install netCDF4 xarray rasterio

In [13]:
pip install cartopy xarray netCDF4


In [14]:
# Import necessary packages
import os
from datetime import datetime
import ftplib
import subprocess
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib.animation import FuncAnimation
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import imageio
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
from IPython.display import HTML
import rasterio
from rasterio.transform import from_origin
from rasterio.warp import calculate_default_transform, reproject, Resampling
from rasterio.crs import CRS
from pyproj import CRS
from osgeo import gdal
import shutil
import re


In [24]:
# # File Name and Path being generated
# file_name = f"APCP_3hourly_ncum_reg_{root_dir}_00Z.nc"
# file_name1 = f"ncumglb_prg_26-31N_79-89E_0.125x0.125_{root_dir}_00Z.nc"

# # Construct the file path
# file_path = os.path.join(root_dir, file_name)
# file_path1 = os.path.join(root_dir, file_name1)

# print(f"File path: {file_path}")
# print(f"File path: {file_path1}")

import os

# Define root directory
root_dir = "20240621"

# Ensure the root directory exists
if not os.path.exists(root_dir):
    os.makedirs(root_dir)

# Generate file names
file_name = f"APCP_3hourly_ncum_reg_{root_dir}_00Z.nc"
file_name1 = f"ncumglb_prg_26-31N_79-89E_0.125x0.125_{root_dir}_00Z.nc"

# Construct the file paths
file_path = os.path.join(root_dir, file_name)
file_path1 = os.path.join(root_dir, file_name1)

# Debugging outputs
print(f"Root directory: {root_dir}")
print(f"Generated file name: {file_name}")
print(f"Generated file path: {file_path}")
print(f"Generated file name1: {file_name1}")
print(f"Generated file path1: {file_path1}")

# Check if directory exists
if not os.path.exists(root_dir):
    print(f"Directory does not exist: {root_dir}")
else:
    # List directory contents
    print(f"Contents of {root_dir}: {os.listdir(root_dir)}")

    # Verify file existence
    if not os.path.exists(file_path):
        print(f"File not found: {file_path}")
    if not os.path.exists(file_path1):
        print(f"File not found: {file_path1}")

    # Ensure files are written
    with open(file_path, 'w') as f:
        f.write('Actual content for APCP_3hourly_ncum_reg')
    with open(file_path1, 'w') as f:
        f.write('Actual content for ncumglb_prg')

    # Verify file creation
    if os.path.exists(file_path):
        print(f"File successfully created: {file_path}")
    if os.path.exists(file_path1):
        print(f"File successfully created: {file_path1}")

    print(f"Contents of {root_dir} after writing files: {os.listdir(root_dir)}")



Root directory: 20240621
Generated file name: APCP_3hourly_ncum_reg_20240621_00Z.nc
Generated file path: 20240621/APCP_3hourly_ncum_reg_20240621_00Z.nc
Generated file name1: ncumglb_prg_26-31N_79-89E_0.125x0.125_20240621_00Z.nc
Generated file path1: 20240621/ncumglb_prg_26-31N_79-89E_0.125x0.125_20240621_00Z.nc
Contents of 20240621: ['ncumglb_prg_26-31N_79-89E_0.125x0.125_20240621_00Z.nc', 'TIFF_Files', 'APCP_3hourly_ncum_reg_20240621_00Z.nc', 'COG_Files']
File successfully created: 20240621/APCP_3hourly_ncum_reg_20240621_00Z.nc
File successfully created: 20240621/ncumglb_prg_26-31N_79-89E_0.125x0.125_20240621_00Z.nc
Contents of 20240621 after writing files: ['ncumglb_prg_26-31N_79-89E_0.125x0.125_20240621_00Z.nc', 'TIFF_Files', 'APCP_3hourly_ncum_reg_20240621_00Z.nc', 'COG_Files']


In [30]:
pip install netCDF4

#### STEP-1: Convert Netcdf to TIFF

In [43]:
# # Function to download and clip NetCDF data to TIFF
# def clip_netcdf_to_tiff(file_path, resolution):
#     # Extract the forecast origin string from the file path
#     filename = os.path.basename(file_path)

#     # Assuming the forecast origin date is in the filename in the format YYYYMMDD
#     forecast_origin_str = filename.split('_')[-2]
#     forecast_origin = datetime.strptime(forecast_origin_str, "%Y%m%d")

#     # Open the NetCDF dataset
#     dataset = xr.open_dataset(file_path)

#     # Subset the dataset to the specified region
#     subset_dataset = dataset.sel(lat=slice(lat_bounds[0], lat_bounds[1]), lon=slice(lon_bounds[0], lon_bounds[1]))

#     # Create the output directory if it doesn't exist
#     output_dir_tiff = os.path.join(root_dir, "TIFF_Files_"+resolution)
#     os.makedirs(output_dir_tiff, exist_ok=True)

#     # List to store file paths
#     tiff_file_paths = []

#     # Define the CRS for the output raster
#     output_crs = CRS.from_epsg(4326)  # WGS 1984

#     # Loop through each time step in the dataset
#     for i, time_step in enumerate(subset_dataset['time']):
#         # Get the data array for the current time step
#         data_array = subset_dataset['param8.1.0'].isel(time=i).values

#         # Get the time string for the current time step
#         time_string = str(time_step.values)[:-10]  # Exclude milliseconds
#         time_format = "%Y-%m-%dT%H:%M:%S"
#         time_obj = datetime.strptime(time_string, time_format)

#         # Calculate the timestep in hours
#         timestep_hours = int((time_obj - forecast_origin).total_seconds() / 3600)
#         timestep_str = f"{timestep_hours:03d}"  # Format as three digits

#         # Calculate the forecast hour index (incrementing integer starting from 00)
#         forecast_hour_index = i + 1
#         forecast_hour_index_str = f"{forecast_hour_index:02d}"  # Format as two digits

#         # Set the output file name
#         output_name = f'NCMRWF_Nepal_{time_obj.strftime("%Y%m%d%H")}F{forecast_hour_index_str}O{forecast_origin_str}00H{timestep_str}R{resolution}.tif'

#         # Get the GeoTransform
#         transform = from_origin(lon_bounds[0], lat_bounds[0], subset_dataset.lon.values[1] - subset_dataset.lon.values[0], subset_dataset.lat.values[0] - subset_dataset.lat.values[1])

#         # Set the output file path
#         output_file = os.path.join(output_dir_tiff, output_name)

#         # Write the data array to the output TIFF file
#         with rasterio.open(output_file, 'w', driver='GTiff', count=1, dtype='float32', crs=output_crs, transform=transform, width=data_array.shape[1], height=data_array.shape[0]) as dst:
#             dst.write(data_array, 1)

#         # Append the output file path to the list
#         tiff_file_paths.append(output_file)

#     print("TIFF files generated and saved successfully.")

#     # Return the list of file paths
#     return tiff_file_paths

# # step-1: Generate TIFF files
# tiff_files = clip_netcdf_
# Define root directory
# root_dir = "20240621"to_tiff(file_path, '4KM')
# tiff_files1 = clip_netcdf_to_tiff(file_path1, '12.5KM')

# # Print TIFF files to verify
# print("TIFF files generated for APCP 4 km:", tiff_files)
# print("TIFF files generated for NCUMGLB 12.5 km:", tiff_files1)

import os
import xarray as xr
from datetime import datetime
from pyproj import CRS
from rasterio.transform import from_origin
import rasterio



# Assuming lat_bounds and lon_bounds are defined
lat_bounds = [26.0, 30.0]
lon_bounds = [80.0, 88.0]

import os

file_path = '/content/20240621/APCP_3hourly_ncum_reg_20240621_00Z.nc'

if os.path.exists(file_path):
    print(f"File exists: {file_path}")

    # Check file size
    file_size = os.path.getsize(file_path)
    print(f"File size: {file_size} bytes")

    # Try to open the file using netCDF4 directly
    try:
        from netCDF4 import Dataset
        nc_file = Dataset(file_path, 'r')
        print(f"Opened file: {file_path}")
        print(f"File dimensions: {nc_file.dimensions.keys()}")
        print(f"File variables: {nc_file.variables.keys()}")
        nc_file.close()
    except Exception as e:
        print(f"Error opening NetCDF file: {e}")
else:
    print(f"File does not exist: {file_path}")


# Function to download and clip NetCDF data to TIFF
def clip_netcdf_to_tiff(file_path, resolution):
    # Extract the forecast origin string from the file path
    filename = os.path.basename(file_path)

    # Assuming the forecast origin date is in the filename in the format YYYYMMDD
    forecast_origin_str = filename.split('_')[-2]
    forecast_origin = datetime.strptime(forecast_origin_str, "%Y%m%d")

    # Open the NetCDF dataset with a specified engine
    dataset = xr.open_dataset(file_path, engine='netcdf4')

    # Subset the dataset to the specified region
    subset_dataset = dataset.sel(lat=slice(lat_bounds[0], lat_bounds[1]), lon=slice(lon_bounds[0], lon_bounds[1]))

    # Create the output directory if it doesn't exist
    output_dir_tiff = os.path.join(root_dir, "TIFF_Files_" + resolution)
    os.makedirs(output_dir_tiff, exist_ok=True)

    # List to store file paths
    tiff_file_paths = []

    # Define the CRS for the output raster
    output_crs = CRS.from_epsg(4326)  # WGS 1984

    # Loop through each time step in the dataset
    for i, time_step in enumerate(subset_dataset['time']):
        # Get the data array for the current time step
        data_array = subset_dataset['param8.1.0'].isel(time=i).values

        # Get the time string for the current time step
        time_string = str(time_step.values)[:-10]  # Exclude milliseconds
        time_format = "%Y-%m-%dT%H:%M:%S"
        time_obj = datetime.strptime(time_string, time_format)

        # Calculate the timestep in hours
        timestep_hours = int((time_obj - forecast_origin).total_seconds() / 3600)
        timestep_str = f"{timestep_hours:03d}"  # Format as three digits

        # Calculate the forecast hour index (incrementing integer starting from 00)
        forecast_hour_index = i + 1
        forecast_hour_index_str = f"{forecast_hour_index:02d}"  # Format as two digits

        # Set the output file name
        output_name = f'NCMRWF_Nepal_{time_obj.strftime("%Y%m%d%H")}F{forecast_hour_index_str}O{forecast_origin_str}00H{timestep_str}R{resolution}.tif'

        # Get the GeoTransform
        transform = from_origin(lon_bounds[0], lat_bounds[1], subset_dataset.lon.values[1] - subset_dataset.lon.values[0], subset_dataset.lat.values[0] - subset_dataset.lat.values[1])

        # Set the output file path
        output_file = os.path.join(output_dir_tiff, output_name)

        # Write the data array to the output TIFF file
        with rasterio.open(output_file, 'w', driver='GTiff', count=1, dtype='float32', crs=output_crs, transform=transform, width=data_array.shape[1], height=data_array.shape[0]) as dst:
            dst.write(data_array, 1)

        # Append the output file path to the list
        tiff_file_paths.append(output_file)

    print("TIFF files generated and saved successfully.")

    # Return the list of file paths
    return tiff_file_paths

# Step-1: Generate TIFF files
tiff_files = clip_netcdf_to_tiff(file_path, '4KM')
tiff_files1 = clip_netcdf_to_tiff(file_path1, '12.5KM')

# Print TIFF files to verify
print("TIFF files generated for APCP 4 km:", tiff_files)
print("TIFF files generated for NCUMGLB 12.5 km:", tiff_files1)


File exists: /content/20240621/APCP_3hourly_ncum_reg_20240621_00Z.nc
File size: 40 bytes
Error opening NetCDF file: [Errno -51] NetCDF: Unknown file format: '/content/20240621/APCP_3hourly_ncum_reg_20240621_00Z.nc'


OSError: [Errno -51] NetCDF: Unknown file format: '/content/20240621/APCP_3hourly_ncum_reg_20240621_00Z.nc'

In [18]:
#Function to combine both the forecast: upto 75 HRS 4Km resolution and upto 121 HRS 12.5Km resolution
def copy_files(src_dir, dst_dir, forecast_threshold=None):
    """
    Copies files from src_dir to dst_dir.
    If forecast_threshold is specified, only copies files with a forecast index >= forecast_threshold.
    """
    os.makedirs(dst_dir, exist_ok=True)

    for filename in os.listdir(src_dir):
        src_file = os.path.join(src_dir, filename)

        if os.path.isfile(src_file):
            if forecast_threshold:
                # Extract the forecast index from the filename
                match = re.search(r'F(\d+)', filename)
                if match and int(match.group(1)) >= forecast_threshold:
                    shutil.copy2(src_file, dst_dir)
            else:
                shutil.copy2(src_file, dst_dir)

# Paths to the source directories
tiff_files_4km = f'{root_dir}/TIFF_Files_4KM'
tiff_files_12_5km = f'{root_dir}/TIFF_Files_12.5KM'

# Path to the destination directory
TIFF_120Hrs = f'{root_dir}/TIFF_Files'

# Step 1: Copy all files from cog_tiff_files_4km
copy_files(tiff_files_4km, TIFF_120Hrs)

# Step 2: Copy only the required files from cog_tiff_files_12_5km
copy_files(tiff_files_12_5km, TIFF_120Hrs, forecast_threshold=26)

# Verify the contents of the new directory
dst_contents = os.listdir(TIFF_120Hrs)
print("Contents of the new directory:", dst_contents)


FileNotFoundError: [Errno 2] No such file or directory: '20240621/TIFF_Files_4KM'

In [ ]:
# Function to plot precipitation data over Nepal from TIFF files
def plot_precipitation_data_tiff(directory):
    # Get a list of all TIFF files in the directory
    tiff_files = [f for f in os.listdir(directory) if f.endswith('.tif')]

    # Define the number of columns and calculate the number of rows
    num_columns = 6
    num_plots = len(tiff_files)
    num_rows = -(-num_plots // num_columns)  # Ceiling division to ensure all images are displayed

    # Set the base figure size to fit an A3 paper while maintaining aspect ratio
    base_fig_size = 11  # Adjust this value to fit within A3 landscape dimensions 42.0 cm (16.53 inches) width x 29.7 cm (11.69 inches) heigh
    fig_width = 12 * num_columns
    fig_height = 6 * num_rows

    # Ensure the figure size fits within estimated size
    max_fig_width = 84
    max_fig_height = 58

    if fig_width > max_fig_width:
        scaling_factor = max_fig_width / fig_width
        fig_width *= scaling_factor


    if fig_height > max_fig_height:
        scaling_factor = max_fig_height / fig_height
        fig_height *= scaling_factor

    # Create subplots with constrained layout
    fig, axes = plt.subplots(num_rows, num_columns, figsize=(fig_width, fig_height), subplot_kw={'projection': ccrs.PlateCarree()}, constrained_layout=True)

    # Flatten the axes array for easy iteration
    axes = axes.flatten()

    # Plot each TIFF file
    for i, tiff_file in enumerate(tiff_files):
        file_path = os.path.join(directory, tiff_file)
        with rasterio.open(file_path) as src:
            precipitation = src.read(1)
            bounds = src.bounds

            ax = axes[i]
            ax.set_extent([lon_bounds[0], lon_bounds[1], lat_bounds[0], lat_bounds[1]], crs=ccrs.PlateCarree())
            ax.add_feature(cfeature.BORDERS.with_scale('10m'), linestyle='-', linewidth=1, edgecolor='red')
            ax.add_feature(cfeature.RIVERS)

            # Define the extent explicitly using bounds
            im = ax.imshow(precipitation, cmap='viridis', extent=[bounds.left, bounds.right, bounds.bottom, bounds.top], transform=ccrs.PlateCarree(), vmin=0, vmax=100)

            # Set the title with the file name
            filename = f'{tiff_file}'

            # Split the filename by underscores to get parts
            parts = filename.split('_')

            # Extract the timestamp part and remove any additional characters
            timestamp = parts[2].split('F')[0]  # 'F' is the separator

            # Extract the forecast index part and remove any additional characters
            forecastIndex = parts[2].split('F')[1].split('O')[0]  # 'F' is the separator

            # Extract the forecast hours part and remove any additional characters
            forecastHrsPart = parts[2].split('H')[1]  # 'H' is the separator
            forecastHrs = forecastHrsPart.split('R')[0]  # 'H' is the separator

            # Use the extracted timestamp and forecast hours as tile
            title = timestamp + 'UTC-F' + forecastIndex + 'H' +forecastHrs

            ax.set_title(title, fontsize=28)

            # Set up gridlines
            gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True, linewidth=2, color='gray', alpha=0.5, linestyle='--')
            gl.top_labels = False  # Hide top labels
            gl.right_labels = False  # Hide right labels
            gl.xformatter = LongitudeFormatter
            gl.yformatter = LatitudeFormatter
            gl.xlabel_style = {'size': 21}
            gl.ylabel_style = {'size': 21}

            # Extract longitude and latitude values for gridlines
            lon_ticks = np.arange(lon_bounds[0], lon_bounds[1] + 1, 2)
            lat_ticks = np.arange(lat_bounds[0], lat_bounds[1] + 1, 2)

            # Set gridline locations
            gl.xlocator = plt.FixedLocator(lon_ticks)
            gl.ylocator = plt.FixedLocator(lat_ticks)

    # Remove any empty subplots
    for j in range(i + 1, len(axes)):
        fig.delaxes(axes[j])

    # Set a common title for all subplots
    fig.suptitle(f"Precipitation Over Nepal (NCMRWF) as forecasted on {root_dir} for 5 Days", fontsize=40)

    # Add a single colorbar for all plots and create a color bar with range 0 to 100
    norm = mcolors.Normalize(vmin=0, vmax=100)
    cbar = fig.colorbar(im, ax=axes, orientation='vertical', pad=0.02, fraction=0.025, norm=norm, location='right')
    cbar.set_label('Precipitation (mm)', fontsize=36)
    cbar.ax.yaxis.set_tick_params(labelsize=28)  # Adjust font size of color bar values

    # Save the figure
    plot_filename = os.path.join(root_dir, f'{root_dir} Precipitation Plot.png')
    plt.savefig(plot_filename, bbox_inches='tight', dpi=600)

    plt.show()

# Call the function to plot precipitation data
plot_precipitation_data_tiff(TIFF_120Hrs)


NameError: name 'TIFF_120Hrs' is not defined

In [ ]:
# Directory to save frames
frames_dir = os.path.join(root_dir, "FRAMES")
os.makedirs(frames_dir, exist_ok=True)

# Function to plot precipitation data over Nepal from TIFF files
def animate_precipitation_data_tiff(directory):
    # Get a list of all TIFF files in the directory
    tiff_files = [f for f in os.listdir(directory) if f.endswith('.tif')]

    # Plot each TIFF file
    for i, tiff_file in enumerate(tiff_files):
        file_path = os.path.join(directory, tiff_file)
        with rasterio.open(file_path) as src:
            precipitation = src.read(1)
            bounds = src.bounds

            fig, ax = plt.subplots(figsize=(12, 8), subplot_kw={'projection': ccrs.PlateCarree()})
            ax.set_extent([lon_bounds[0], lon_bounds[1], lat_bounds[0], lat_bounds[1]], crs=ccrs.PlateCarree())
            ax.add_feature(cfeature.BORDERS.with_scale('10m'), linestyle='-', linewidth=1, edgecolor='red')
            ax.add_feature(cfeature.RIVERS)

            # Define the extent explicitly using bounds
            im = ax.imshow(precipitation, cmap='viridis', extent=[bounds.left, bounds.right, bounds.bottom, bounds.top], transform=ccrs.PlateCarree(), vmin=0, vmax=100)

            # Set the title with the file name
            filename = f'{tiff_file}'

            # Split the filename by underscores to get parts
            parts = filename.split('_')

            # Extract the timestamp part and remove any additional characters
            timestamp = parts[2].split('F')[0]  # 'F' is the separator

            # Extract the forecast index part and remove any additional characters
            forecastIndex = parts[2].split('F')[1].split('O')[0]  # 'F' is the separator

            # Extract the forecast hours part and remove any additional characters
            forecastHrsPart = parts[2].split('H')[1]  # 'H' is the separator
            forecastHrs = forecastHrsPart.split('R')[0]  # 'H' is the separator

            # Use the extracted timestamp and forecast hours as tile
            title = timestamp + 'UTC-F' + forecastIndex + 'H' +forecastHrs

            ax.set_title(title, fontsize=18, pad=30)

            # Set up gridlines
            gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True, linewidth=2, color='gray', alpha=0.5, linestyle='--')
            gl.top_labels = False  # Hide top labels
            gl.right_labels = False  # Hide right labels
            gl.xformatter = LongitudeFormatter()
            gl.yformatter = LatitudeFormatter()
            gl.xlabel_style = {'size': 14}
            gl.ylabel_style = {'size': 14}

            # Extract longitude and latitude values for gridlines
            lon_ticks = np.arange(lon_bounds[0], lon_bounds[1] + 1, 2)
            lat_ticks = np.arange(lat_bounds[0], lat_bounds[1] + 1, 2)

            # Set gridline locations
            gl.xlocator = plt.FixedLocator(lon_ticks)
            gl.ylocator = plt.FixedLocator(lat_ticks)

            # Set a common title for all subplots
            fig.suptitle(f"Precipitation Over Nepal (NCMRWF) as forecasted on {root_dir}", fontsize=20, y=0.90)

            # Add a single colorbar for the plot and create a color bar with range 0 to 100
            norm = mcolors.Normalize(vmin=0, vmax=100)
            cbar = fig.colorbar(im, ax=ax, orientation='vertical', pad=0.07, fraction=0.025, norm=norm, location='right')
            cbar.set_label('Precipitation (mm)', fontsize=14)
            cbar.ax.yaxis.set_tick_params(labelsize=12)  # Adjust font size of color bar values

            # Save each frame
            frame_filename = os.path.join(frames_dir, f'frame_{i:04d}.png')
            plt.savefig(frame_filename, bbox_inches='tight', dpi=400)
            plt.close(fig)

    print("Frames saved successfully.")

     # Create an animated GIF from the frames with a delay between frames
    frame_files = sorted([os.path.join(frames_dir, f) for f in os.listdir(frames_dir) if f.endswith('.png')])
    gif_filename = os.path.join(root_dir, f'precipitation_animation_{root_dir}.gif')
    with imageio.get_writer(gif_filename, mode='I', duration=1) as writer:  # 1-second delay
        for frame_file in frame_files:
            image = imageio.v2.imread(frame_file)
            writer.append_data(image)

    print(f"Animation saved as {gif_filename}.")

    # Function to update each frame
    def update(frame):
        im.set_data(plt.imread(frame_files[frame]))
        return im,

    # Create a figure without the outer frame
    fig, ax = plt.subplots(figsize=(10, 8))
    ax.axis('off')  # Turn off the axis

    # Display the first frame
    im = ax.imshow(plt.imread(frame_files[0]))

    # Create the animation with the specified interval
    ani = FuncAnimation(fig, update, frames=len(frame_files), interval=interval_time)

    # Display the animation
    return HTML(ani.to_html5_video())

# Call the function to plot precipitation data
animate_precipitation_data_tiff(TIFF_120Hrs)


#### STEP-2: Convert Tiff to COG

In [19]:
# Get all TIFF files in the directory
tiff_files = [os.path.join(TIFF_120Hrs, f) for f in os.listdir(TIFF_120Hrs) if f.endswith('.tif')]

# Print the list of TIFF files
print(f"TIFF files generated: {tiff_files}")

# Function to convert TIFF to COG TIFF
def tiff_to_cogtiff(tiff_file_paths):
    # Create the output directory if it doesn't exist
    output_dir_cog = os.path.join(root_dir, "COG_Files")
    os.makedirs(output_dir_cog, exist_ok=True)

    # List to store COG file paths
    cog_file_paths = []

    if not tiff_file_paths:
        print("No TIFF files provided for conversion.")
        return cog_file_paths

    for tiff_file in tiff_file_paths:
        # Set the COG output file path
        cog_output_file = os.path.join(output_dir_cog, os.path.basename(tiff_file).replace('.tif', 'C.tif'))

        # Print debug information
        print(f"Converting {tiff_file} to {cog_output_file}")

        # Convert the TIFF to Cloud Optimized GeoTIFF
        try:
            result = subprocess.run(
                ['gdal_translate', tiff_file, cog_output_file, '-co', 'TILED=YES', '-co', 'COPY_SRC_OVERVIEWS=YES', '-co', 'COMPRESS=DEFLATE'],
                check=True,
                capture_output=True,
                text=True
            )
            print(f"GDAL output: {result.stdout}")
            print(f"GDAL errors: {result.stderr}")

            # Append the COG file path to the list
            cog_file_paths.append(cog_output_file)
        except subprocess.CalledProcessError as e:
            print(f"Error converting {tiff_file}: {e.stderr}")

    print("COG TIFF files generated and saved successfully.")

    return cog_file_paths

# Step 3: Convert the TIFF files to COG TIFF
cog_tiff_files = tiff_to_cogtiff(tiff_files)

# Print TIFF files to verify
print("Cloud Optimized GeoTIFFs files generated:", cog_tiff_files)


TIFF files generated: []
No TIFF files provided for conversion.
Cloud Optimized GeoTIFFs files generated: []
